In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

In [2]:
df_orig = pd.read_csv(r'C:\Users\bhavy\Desktop\INFO_DS\Project3_Data\lending-club-loan-data\loan.csv', low_memory =False)
pd.set_option('display.max_columns',None)
df_orig.fillna(0,inplace=True)

df = df_orig

# Drop unneccesary columns
df = df.drop(['url', 'desc', 'policy_code', 'last_pymnt_d', 'next_pymnt_d', 'earliest_cr_line', 'emp_title'], axis=1)
df = df.drop(['id', 'title', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'zip_code'], axis=1)

i = len(df)
df = pd.DataFrame(df[df['loan_status'] != "Does not meet the credit policy. Status:Fully Paid"])
df = pd.DataFrame(df[df['loan_status'] != "Does not meet the credit policy. Status:Charged Off"])
df = pd.DataFrame(df[df['loan_status'] != "Issued"])
df = pd.DataFrame(df[df['loan_status'] != "In Grace Period"])
a = len(df)
print(f"We dropped {i-a} rows, a {((i-a)/((a+i)/2))*100}% reduction in rows")


We dropped 17462 rows, a 1.987371507133687% reduction in rows


In [4]:
cols_to_keep = ['loan_amnt', 'dti','addr_state','emp_length','purpose','annual_inc']
train, test = train_test_split(df, train_size = 0.7)
train_y = train['int_rate']
test_y = test['int_rate']
train_x = train[cols_to_keep]
test_x = test[cols_to_keep]

C:\Users\bhavy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# Discreet value integer encoder
label_encoder = preprocessing.LabelEncoder()
# State is string and we want discre integer values
train_x['state'] = label_encoder.fit_transform(train_x['addr_state'])
test_x['state'] = label_encoder.fit_transform(test_x['addr_state'])
train_x['purp'] = label_encoder.fit_transform(train_x['purpose'])
test_x['purp'] = label_encoder.fit_transform(test_x['purpose'])

C:\Users\bhavy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\bhavy\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\bhavy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [6]:
train_x = train_x._get_numeric_data()
test_x = test_x._get_numeric_data()
train_x.head()

,loan_amnt,dti,annual_inc,state,purp
709148,10000.0,25.83,56600.0,9,2
623808,20000.0,9.41,55000.0,6,2
346816,17500.0,23.80,106000.0,22,2
820778,30000.0,24.59,90000.0,43,2
17272,17625.0,24.72,55000.0,14,1


In [18]:
train_y_enc = label_encoder.fit_transform(train_y)
test_y_enc = label_encoder.fit_transform(test_y)

In [ ]:


#logistic regression
log_reg = LogisticRegression()
log_reg.fit(train_x, train_y_enc)



C:\Users\bhavy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\bhavy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
print ("Intercept is ",log_reg.intercept_)
print("Coefficient is ",log_reg.coef_)
y_pred=log_reg.predict(test_x)

In [15]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [16]:
def calculate_roc_curve(Y_test, y_pred, pos_label):    
    fpr, tpr, _ = roc_curve(Y_test, preds)
    #Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1-Specificity')
    plt.ylabel('Sensitivity')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

def calculate_confusion_matrix(y_true, y_pred):    
    cm=confusion_matrix(y_true, y_pred)
    print(cm)

In [12]:
from sklearn.metrics import accuracy_score

In [ ]:
#calculate ROC curve
preds = log_reg.predict_proba(test_x)[:,1]
calculate_roc_curve(test_y, preds,2)

#calculate Confusion Matrix
calculate_confusion_matrix(test_y, y_pred)

print(accuracy_score(test_y, y_pred))

In [10]:
from sklearn.ensemble import RandomForestClassifier

#Random Forest
rf = RandomForestClassifier(n_jobs=2)
rf.fit(train_x, train_y_enc)


C:\Users\bhavy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
preds = rf.predict_proba(test_x)[:,1]
y_pred=rf.predict(test_x)
#calculate ROC curve
calculate_roc_curve(test_y_enc, y_pred,2) 
 #calculate Confusion Matrix

calculate_confusion_matrix(test_y_enc, y_pred)
print(accuracy_score(test_y_enc, y_pred))

ValueError: multiclass format is not supported